# **Limpieza *Lamudi***
PROCESO DE ESTANDARIZACIÓN DE DATOS PARA OBTENCIÓN DE BASE **DATALPINE**

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos
from datetime import datetime, date  # Para trabajar con fechas y horas
from currency_converter import CurrencyConverter # convertidor de monedas
from dateutil.relativedelta import relativedelta  # Para calcular diferencias entre fechas, incluyendo años, meses y días
# Mapear coincidencia
from fuzzywuzzy import process
from fuzzywuzzy import fuzz 
# Instalar python-Levenshtein para mejorar el rendimiento de SequenceMatcher
import Levenshtein

In [2]:
# ESTILO DE IMPRESIONES

# Colores ANSI
BLACK = "\033[90m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
MAGENTA = "\033[95m"
CYAN = "\033[96m"
WHITE = "\033[97m"
RESET = "\033[0m"  # Restablecer el color por defecto
# Estilos de texto
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
BLINK = "\033[5m"
REVERSE = "\033[7m"  # Invertir colores (fondo y texto)

# 1. Importación de datos

Definir la ruta relativa al archivo, incluyendo la extensión

In [3]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo, incluye extensión
#archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\ciudades\\tulancingo\\clean\\marzo_2024.csv"
archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\data\\Vallarta\\Original\\lamudi-Bahia-de-Balderas_Inmuebles_Septiembre 2024.csv"
archivo_dir = os.path.join(directorio_actual)
archivo_path = os.path.join(archivo_dir, archivo_filename)

Elige el tipo de archivo a leer

In [ ]:
# Intentar leer el archivo con codificación utf-8, si falla, intentar con latin-1
try:
    df = pd.read_csv(archivo_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(archivo_path, encoding='latin-1')
df.info()

Limpieza de nulos
Sólo en caso de que el scraping no haya llenado los datos faltantes correctamente.

In [5]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'denominacion': ''}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
#df.fillna({'tiempo_de_publicacion': ''}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

# 2. Duplicados

* Estandarizar descripción
* Eliminación de los duplicados, dejando uno de los resultantes

In [6]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['descripcion'] = df['descripcion'].astype(str)
# Convertir a minúsculas
df['descripcion'] = df['descripcion'].str.lower()
# Eliminar acentos y carácteres especiales
import unicodedata 
def eliminar_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['descripcion'] = df['descripcion'].apply(eliminar_acentos)
df['descripcion'] = df['descripcion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
inicio = df.shape

## a) Con todos los datos iguales

In [ ]:
duplicados = df.loc[df.duplicated()]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

In [ ]:
df = df.drop_duplicates(keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

In [ ]:
duplicados = df.loc[df.duplicated()]
print(duplicados)

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Descripción y precio iguales

In [ ]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

In [ ]:
df = df.drop_duplicates(subset=['descripcion','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

In [ ]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
print(duplicados)

## c) M2 construcción, Ubicación y Precio

In [ ]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

In [ ]:
df = df.drop_duplicates(subset=['metros_construido','ubicacion','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

In [ ]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
print(duplicados)

## d) Características y Precio

In [ ]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

In [ ]:
df = df.drop_duplicates(subset=['estacionamientos','recamaras','banos','medio_banos','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

In [ ]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
print(duplicados)

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

# 3. Manipulación de precios

## a) Precio en descripción

Manten las diferentes formas de que el precio se encuentre en la descripción

In [14]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\$?\s*(\d{1,3}(?:[.,]?\d{3})*(?:[.,]\d{2})?)\s*(mxn|usd|pesos|monto|precio|costo)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '')#.replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)
inicio = df.shape

In [15]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['precio'] = f"{MAGENTA}{row['precio']}{RESET}"
    row_colored['precio_desc'] = f"{GREEN}{row['precio_desc']}{RESET}"
    return row_colored

### Precio [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

### Precio [0/0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")
print("Proceso completado.")

### UMAS

In [ ]:
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )]
# Define price ranges based on 'metros_total'
def get_price_range(metros_total):
    if metros_total <= 40:
        return 118
    elif metros_total <= 50:
        return 150
    elif metros_total <= 71:
        return 275
    elif metros_total <= 102:
        return 550
    elif metros_total <= 156:
        return 1125
    else:
        return 1500

# Assign a price range to each row and convert it to MXN
totales['precio'] = totales['metros_total'].apply(get_price_range) * 86.88



Eliminar registros que no recuperaron su precio ni desde la descripción

In [18]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

### Precio [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input(f"Elige una opción (1, 2, 3) para el registro {idx}: ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Denominación MXN

Si existe una denominación que no sea mxn, se convierta el precio

In [17]:
#df['denominacion'] = 'MXN'

In [ ]:
df['precio'].isna().sum()

In [ ]:
import datetime
c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'))
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
df['fecha_conversion'] = fecha_conversion
print(df[['precio', 'denominacion', 'precio_mxn', 'precio_usd', 'fecha_conversion']].head())

In [21]:
#df['precio'] = df['precio_mxn']

## c) Precio fuera del estandar

Identificar y eliminar propiedades que cuenten con precios bajos y altos al estandar

In [ ]:
inicio = df.shape
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
print(df['precio'].sort_values())
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## D) Segmento o categoría

In [ ]:
# Crear un diccionario donde las llaves son las categorías y los valores son las tuplas de rangos
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

# Asignar la categoría a cada registro
df['categoria'] = df['precio'].apply(asignar_categoria)
print(df['categoria'].unique())

# 4. Meses transcurridos

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Convertir el mes abreviado en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Caso 1: formato "20 abr. 2023" -> "2023-04-20"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month.replace('.', ''))  # Eliminar puntos si existen
        return f"{year}-{month_int:02d}-{int(day):02d}"
    except (ValueError, AttributeError):
        pass
    # Caso 2: formato "19-abr-24" -> "2019-04-24"
    try:
        day, month, year = date_str.split('-')
        year = '20' + year if int(year) < 50 else '19' + year
        month_int = month_to_int(month)  # No es necesario eliminar puntos
        return f"{year}-{month_int:02d}-{int(day):02d}"
    except (ValueError, AttributeError):
        pass
    # Caso 3: formato "YYYY-MM-DD HH:MM:SS" o "YYYY-MM-DD"
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Si no se puede convertir, devolver None
        return None

# Aplicar la función a la columna 'tiempo_de_publicacion'
df['fecha_estandarizada'] = df['tiempo_de_publicacion'].apply(standardize_date)

# Convertir a datetime y manejar errores
df['fecha_estandarizada'] = pd.to_datetime(df['fecha_estandarizada'], errors='coerce')

# Obtener la fecha más actual de la base original
fecha_mas_actual = df['fecha_estandarizada'].max()

# Función para calcular la diferencia en días y meses
def calcular_dias_y_meses(fecha):
    if pd.isnull(fecha):
        return None, None
    rd = relativedelta(fecha_mas_actual, fecha.date())
    # Redondear los meses a entero
    meses_redondeados = rd.years * 12 + rd.months
    # Redondear los días a entero
    dias_redondeados = rd.days + (rd.months * 30) + (rd.years * 365)
    return dias_redondeados, meses_redondeados

# Aplicar la función para calcular los días y meses transcurridos
df['dias_transcurridos'], df['meses_transcurridos'] = zip(*df['fecha_estandarizada'].apply(calcular_dias_y_meses))

#Registro de fecha de conversión
df['fecha_mas_actual'] = fecha_mas_actual

# Mostrar el DataFrame con las fechas estandarizadas, días y meses transcurridos
print(df[['tiempo_de_publicacion', 'fecha_estandarizada', 'dias_transcurridos', 'meses_transcurridos', 'fecha_mas_actual']])

In [ ]:
print(df[['tiempo_de_publicacion', 'fecha_estandarizada', 'dias_transcurridos', 'meses_transcurridos', 'fecha_mas_actual']].iloc[4])

# 5. Tipo de propiedad

## a) Estandarizar

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['tipo'] = df['tipo'].astype(str)
# Convertir a minúsculas
df['tipo'] = df['tipo'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['tipo'] = df['tipo'].apply(eliminar_acentos)
# Convertir la primera letra de cada palabra a mayúscula
df['tipo'] = df['tipo'].str.title()
df['tipo'].unique() 

## b) Lista de tipos deseados

* Casas: Casa en condominio, 
* Departamento: Penthouse, Loft

In [ ]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa En Condominio|Casa En Fraccionamiento|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

## c) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

Desde el nombre

In [ ]:
# Filtrar los registros según las condiciones dadas
totales =df[df['propiedad'].str.contains('terreno|lote|parcela', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['propiedad']])
    print(row[['descripcion']])
    print(row[['url']])
    print(f"{GREEN}\n¿Te gustaría eliminar el registro?{RESET}")
    # Solicitar la opción al usuario
    option = input("Elige una opción: Si(1) / No(0)): ").strip().lower()
    
    if option == '1':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

#### Desde descripción

In [ ]:
totales =df[df['descripcion'].str.contains('lote', case=False) & ~df['propiedad'].str.contains('casa', case=False)]
totales.shape

In [ ]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lote', case=False) & ~df['propiedad'].str.contains('casa', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():    
    print(f"Registro {idx}:")
    print(row[['descripcion']])
    print(row[['url']])
    print(f"{GREEN}\n¿Te gustaría eliminar el registro?{RESET}")
    # Solicitar la opción al usuario
    option = input("Elige una opción: Si(1) / No(0)): ").strip().lower()
    
    if option == '1':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

# 6. Metros totales (terreno)

## a) Metros en descripción

In [26]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales|mt2)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)
inicio = df.shape

In [27]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['metros_total'] = f"{MAGENTA}{row['metros_total']}{RESET}"
    row_colored['totales_desc'] = f"{GREEN}{row['totales_desc']}{RESET}"
    return row_colored

### Metros [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_total', 'totales_desc', 'url']])
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

### Metros [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_total', 'totales_desc', 'url']])
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")    
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

AQUÍ DEBERÍA MANEJAR LAS PROPIEDADES QUE QUEDARON CON 0, PARA SUSTITUIRLAS DE ACUERDO A LOS M2 CONSTRUIDOS

## b) Manejo por rangos de UMA

In [ ]:
print(f"{RED}Paso opcional{RESET}")

Paso opcional por si no se espera cambiar el precio por el que se tenga en descripción

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

Cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
totales = df.apply(actualizar_metros_total, axis=1)
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

#df = df.apply(actualizar_metros_total, axis=1)
print(totales[['metros_total', 'totales_desc', 'url']])

Sólo si aceptas los cambios

In [ ]:
df = df.apply(actualizar_metros_total, axis=1)

# 7. M2 construidos

## a) M-construidos en descripción

In [30]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)
inicio = df.shape

In [31]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['metros_construido'] = f"{MAGENTA}{row['metros_construido']}{RESET}"
    row_colored['construidos_desc'] = f"{GREEN}{row['construidos_desc']}{RESET}"
    return row_colored

### Metros [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

### Metros [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

# 7.5 Terreno igual al construido cuando no se tiene

In [34]:
# Esta línea de código reemplaza los valores faltantes en la columna 'metros_total' con el valor de la columna 'metros_construido'.
df['metros_total'] = df['metros_total'].fillna(df['metros_construido'])

# 8. Precio / m2

## M2 Totales (terreno)

In [35]:
# Calcular precio por metro cuadrado del terreno
#df['precio_m2_terreno'] = df['precio']/df['metros_total']
df['precio_m2_terreno'] = df['precio_mxn']/df['metros_total']

## M2 Construidos

In [36]:
# Calcular precio por metro cuadrado de la cosntrucción
#df['precio_m2_construido'] = df['precio']/df['metros_construido']
df['precio_m2_construido'] = df['precio_mxn']/df['metros_construido']

# 9. Estacionamientos

## a) Estacionamientos descritos

Asigna las diferentes maneras en las que puede encontrarse descrito el espacio de estacionamientos

In [37]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)
inicio = df.shape

In [38]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['estacionamientos'] = f"{MAGENTA}{row['estacionamientos']}{RESET}"
    row_colored['estacionamiento_desc'] = f"{GREEN}{row['estacionamiento_desc']}{RESET}"
    return row_colored

### Estacionamientos [0 / #]

In [ ]:
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 
totales.shape

In [45]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) & (df['estacionamiento_desc'] < 7) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])

    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Sustituir el valor manualmente")
    print("Enter, para mantener el mínimo de 1")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        #Mínimo 1
        new_value = 1
        df.at[idx, 'estacionamientos'] = float(new_value)
        print("Mínimo de 1. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Mínimo de 1. Pasando al siguiente registro...

---

Registro 707:
estacionamientos                                                                             0
estacionamiento_desc                                                                       5.0
url                     https://www.lamudi.com.mx/detalle/41032-73-e769349901ae-c9e4-7fef420d-b95b-4666
Name: 707, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'
3. Sustituir el valor manualmente
Enter, para mantener el mínimo de 1
Mínimo de 1. Pasando al siguiente registro...

---

Registro 712:
estacionamientos                                                                             0
estacionamiento_desc                                                                       2.0
url                     https://www.lamudi.com.mx/detalle/41032-73-139147de13c9-a850-ef15874f-af35-380b
Name: 712, dtype: object

¿Qué acción te g

In [47]:
df.loc[(df['estacionamientos'] == 0) | ((df['estacionamiento_desc'] > 13) | df['estacionamiento_desc'].isna()), 'estacionamientos'] = 1

In [48]:
df['estacionamientos'].value_counts()

estacionamientos
1    1556
2      44
4       6
6       2
3       2
Name: count, dtype: int64

### Estacionamientos [0 / 0]

In [ ]:
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
totales.shape

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = int(new_value)
    #elif option == '2':
        # Sustituir el valor de 'estacionamientos' por 'estacionamiento_desc'
        #df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
         # Sustituir por 1, como mínimo
        new_value = 1
        df.at[idx, 'estacionamientos'] = int(new_value)
        print("Valor de 1, Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

### Estacionamientos [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [65]:
df = df[df['estacionamientos'] <= 10]
#df[['estacionamientos','estacionamiento_desc','url']]

# 10. Recámaras

## a) Recámaras en descripción

In [49]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)
inicio = df.shape

In [50]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['recamaras'] = f"{MAGENTA}{row['recamaras']}{RESET}"
    row_colored['recamaras_desc'] = f"{GREEN}{row['recamaras_desc']}{RESET}"
    return row_colored

### Recámaras [0 / 0]

In [51]:
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
totales.shape

(18, 30)

In [53]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['url']])
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '1':
        new_value = 1
        df.at[idx, 'recamaras'] = float(new_value)
        # Pasar al siguiente registro
        print("Mínimo de 1")
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Registro 680:
recamaras                                                                             0
recamaras_desc                                                                      nan
url               https://www.lamudi.com.mx/detalle/41032-73-1f20c8552308-c4dd-88d3400-8f13-4f84
Name: 680, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
3. Sustituir el valor manualmente



---

Registro 699:
recamaras                                                                              0
recamaras_desc                                                                       nan
url               https://www.lamudi.com.mx/detalle/41032-73-b261db097f1e-9402-f679b864-a83e-4ee1
Name: 699, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
3. Sustituir el valor manualmente

---

Registro 758:
recamaras                                                                              0
recamaras_desc                                                                       nan
url               https://www.lamudi.com.mx/detalle/41032-73-5f5d25bc6650-966d-f8a274e6-8db8-3932
Name: 758, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
3. Sustituir el valor manualmente

---

Registro 785:
recamaras                                                                             0
recamaras_desc                                    

### Recámaras [0 / #]

In [54]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '1':
        new_value = 1
        df.at[idx, 'recamaras'] = float(new_value)
        print("Mínimo de 1")
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 984:
recamaras                                                                              0
recamaras_desc                                                                       2.0
url               https://www.lamudi.com.mx/detalle/41032-73-a94557336877-f44b-e092e134-b1fa-4a54
Name: 984, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'
3. Sustituir el valor manualmente

---

Registro 1027:
recamaras                                                                              0
recamaras_desc                                                                       3.0
url               https://www.lamudi.com.mx/detalle/41032-73-74e7b7be995c-6d5c-9386fc5f-b653-3c46
Name: 1027, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'
3. Sustituir el valor manualmente

---

Registro 1028:
recamaras    

### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Mantener rango de recámaras <= 7 

In [ ]:
print(df['recamaras'].unique())
df = df[(df['recamaras'] <= 10)] 
#print(df[['recamaras','recamaras_desc','url']])

# 11. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [55]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

In [56]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['banos'] = f"{MAGENTA}{row['banos']}{RESET}"
    row_colored['baños_completos'] = f"{GREEN}{row['baños_completos']}{RESET}"
    return row_colored
inicio = df.shape

Poner como mínimo para el registro 1 en baños

#### [0 / 0]

In [57]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '1':
        continue  # Pasar al siguiente registro
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [58]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    print("3. Sustituir el valor manualmente")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    elif option=='1':
        continue
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 219:
banos                                                                                 0
baños_completos                                                                       0
url                https://www.lamudi.com.mx/detalle/41032-73-9f02e696fd59-660-18ffac5-adf1-76ca
Name: 219, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Sustituir el valor manualmente

---

Registro 281:
banos                                                                                 0
baños_completos                                                                       0
url                https://www.lamudi.com.mx/detalle/41032-73-d21c93e34c4-406-5227728e-8455-304d
Name: 281, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Sustituir el valor manualmente

---

Registro 320:
banos                       

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    print("3. Sustituir el valor manualmente")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    elif option == '1':
        continue
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

### Medio baños

In [59]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['medio_banos'] = f"{MAGENTA}{row['medio_banos']}{RESET}"
    row_colored['medios_baños'] = f"{GREEN}{row['medios_baños']}{RESET}"
    return row_colored
inicio = df.shape

#### [0 / 0]

In [60]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_banos'] == 0) & (df['medios_baños'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [61]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_banos']==0) & (df['medios_baños']!=0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'medio_banos' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_banos' por 'medios_baños'
        df.at[idx, 'medio_banos'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_banos']!=0) & (df['medios_baños'].notna())) & (df['medio_banos'] != df['medios_baños']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'medio_banos' por el de 'medios_baños'")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_banos'] = df.at[idx, 'medios_baños']
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## B) Total de baños

In [62]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      3            0         3.0
1      4            1         4.5
2      3            0         3.0
3      2            0         2.0
4      5            0         5.0


# 12. Ubicación

## a) Desde descripción

In [60]:
# Función para extraer ubicación de la descripción
def extraer_ubicacion(descripcion):
    descripcion = descripcion.lower()
    # Patrones para buscar ubicaciones comunes
    patrones = [
        r'(?:ubicad[oa] en|en) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:calle|avenida|av\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:colonia|col\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:fraccionamiento|fracc\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})'
    ]
    2.
    for patron in patrones:
        match = re.search(patron, descripcion)
        if match:
            return match.group(1).strip().title()
    
    return None

# Aplicar la función al DataFrame
df['ubicacion_desc'] = df['descripcion'].apply(extraer_ubicacion)

#### Cuando la ubicación dice centro:

In [ ]:
# Filtrar los registros donde la ubicación contiene la palabra "centro"
df_centro = df[df['ubicacion'].str.contains('centro|Centro', case=False, na=False)]
num_registros_centro = len(df_centro)

print(f"Número de registros que contienen 'centro' en la ubicación:",f"{RED} {num_registros_centro}{RESET}")

In [46]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
    row_colored['ubicacion_desc'] = f"{GREEN}{row['ubicacion_desc']}{RESET}"
    return row_colored
inicio = df.shape

In [ ]:

pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in df_centro.iterrows():
    print(f"Registro {idx}:")    
    #print(row[['ubicacion', 'ubicacion_desc', 'url']])
    print(colorize_row(row)[['ubicacion', 'ubicacion_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'ubicacion': ")
        df.at[idx, 'ubicacion'] = new_value  
    elif option == '2':
        # Sustituir el valor de 'ubicacion' por 'ubicacion_desc'
        df.at[idx, 'ubicacion'] = df.at[idx, 'ubicacion_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [63]:
# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = location.split(',')
    parts = [part.strip() for part in parts]  # Eliminar espacios al principio y final de cada parte
    parts = [unicodedata.normalize('NFKD', part).encode('ascii', 'ignore').decode('utf-8') for part in parts]  # Eliminar acentos y otros caracteres no ascii
    parts += [None] * (3 - len(parts))  # Asegurar que siempre hay 3 partes
    return parts[:3]  # Devolver solo las primeras 3 partes

df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiando valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

,ubicacion,Colonia,Municipio,Estado
0,"Mezcales, Bahía de Banderas, Nayarit",Mezcales,Bahia de Banderas,Nayarit
1,"Las Jarretaderas, Bahía de Banderas, Nayarit",Las Jarretaderas,Bahia de Banderas,Nayarit
2,"Bucerías Centro, Bahía de Banderas, Nayarit",Bucerias Centro,Bahia de Banderas,Nayarit
3,"San Francisco, Bahía de Banderas, Nayarit",San Francisco,Bahia de Banderas,Nayarit
4,"Cruz de Juanacaxtle, Bahía de Banderas, Nayarit",Cruz de Juanacaxtle,Bahia de Banderas,Nayarit
...,...,...,...,...
1642,"Valle de Banderas, Bahía de Banderas, Nayarit",Valle de Banderas,Bahia de Banderas,Nayarit
1643,"Mezcales, Bahía de Banderas, Nayarit",Mezcales,Bahia de Banderas,Nayarit
1644,"Valle de Banderas, Bahía de Banderas, Nayarit",Valle de Banderas,Bahia de Banderas,Nayarit
1645,"Rincón del Cielo, Bahía de Banderas, Nayarit",Rincon del Cielo,Bahia de Banderas,Nayarit


In [64]:
import pandas as pd
import unicodedata
# Función para normalizar y limpiar la ubicación
def normalize_text(text):
    text = text.strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = [normalize_text(part) for part in location.split(',')]
    if len(parts) == 1:  # No hay comas, intentar inferir
        # Suponiendo que 'queretaro queretaro' o 'el marqués el marqués' está al final y es el estado y municipio
        if location.lower().endswith('queretaro queretaro'):
            return [location.replace('queretaro queretaro', ''), 'queretaro', 'queretaro']
        elif location.lower().endswith('el marques queretaro'):
            return [location.replace('el marques queretaro', ''), 'el marques', 'queretaro']
        elif location.lower().endswith('juriquilla querataro arteaga'):
            return [location.replace('juriquilla querataro arteaga', ''), 'juriquilla', 'querataro arteaga']
        elif location.lower().endswith('querataro querataro arteaga'):
            return [location.replace('querataro querataro arteaga', ''), 'querataro', 'querataro arteaga']
        
        elif location.lower().endswith('pachuca de soto hidalgo'):
            return [location.replace('pachuca de soto hidalgo', ''), 'pachuca de soto', 'hidalgo']
        elif location.lower().endswith('mineral de la reforma hidalgo'):
            return [location.replace('pmineral de la reforma hidalgo', ''), 'mineral de la reforma', 'hidalgo']
        
        
        else:
            # Solicitar entrada manual si no se puede inferir
            colonia = input(f"Introduce Colonia para '{location}': ").strip() or 'Sin_colonia'
            municipio = input("Introduce Municipio (dejar vacío para 'Sin_municipio'): ").strip() or 'Sin_municipio'
            estado = input("Introduce Estado (dejar vacío para 'Sin_estado'): ").strip() or 'Sin_estado'
            return [colonia, municipio, estado]
    # Asegurar siempre 3 partes
    parts += [None] * (3 - len(parts))
    return parts[:3]

# Aplicar función al DataFrame
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiar valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')

# Mostrar resultados
print(df[['ubicacion', 'Colonia', 'Municipio', 'Estado']])

                                            ubicacion              Colonia  \
0                Mezcales, Bahía de Banderas, Nayarit             Mezcales   
1        Las Jarretaderas, Bahía de Banderas, Nayarit     Las Jarretaderas   
2         Bucerías Centro, Bahía de Banderas, Nayarit      Bucerias Centro   
3           San Francisco, Bahía de Banderas, Nayarit        San Francisco   
4     Cruz de Juanacaxtle, Bahía de Banderas, Nayarit  Cruz de Juanacaxtle   
...                                               ...                  ...   
1642    Valle de Banderas, Bahía de Banderas, Nayarit    Valle de Banderas   
1643             Mezcales, Bahía de Banderas, Nayarit             Mezcales   
1644    Valle de Banderas, Bahía de Banderas, Nayarit    Valle de Banderas   
1645     Rincón del Cielo, Bahía de Banderas, Nayarit     Rincon del Cielo   
1646     Jardines del Sol, Bahía de Banderas, Nayarit     Jardines del Sol   

              Municipio   Estado  
0     Bahia de Banderas  Nay

In [ ]:
nan_locations = df[(df['Colonia'].isna() | df['Colonia'].eq('Sin_colonia')) & 
                    (df['Municipio'].isna() | df['Municipio'].eq('Sin_municipio')) & 
                    (df['Estado'].isna() | df['Estado'].eq('Sin_estado'))]
nan_locations

In [ ]:
incompletos.columns

In [65]:
incompletos = df[df['Municipio'].eq('Sin_municipio')]
#incompletos['ubicacion'] = 'Fraccionamiento Vilanova Nuevo Vallarta'
incompletos[['ubicacion','Colonia','Estado','Municipio']]

,ubicacion,Colonia,Estado,Municipio


In [67]:
df = df.drop(incompletos.index)

In [ ]:
incompletos = df[df['Municipio'].eq('Sin_municipio')]
for index, row in incompletos.iterrows():
    ubicacion = row['ubicacion']
    if 'puerto vallarta' in ubicacion.lower():
        df.loc[index, 'Municipio'] = 'Puerto Vallarta'
print(incompletos)

Solo en caso de que se tengan vacíos en las columnas resultantes

In [67]:
#df['Municipio'] = df['Municipio'].str.replace('BahAa de Banderas', 'Bahia de Banderas')
df['Municipio'] = df['Municipio'].str.replace('Nayarit', 'Bahia de Banderas')
print(df['Municipio'].unique())


['Bahia de Banderas']


In [ ]:
print(df['Municipio'].unique())
df['Municipio'] = df['Municipio'].str.replace('Querétaro', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('juriquilla', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('Juriquilla', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('El Campanario', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('Santiago de Queretaro', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('querataro', 'queretaro')
df['Municipio'] = df['Municipio'].str.replace('Queretaro', 'queretaro')

df['Municipio'] = df['Municipio'].str.replace('Hidalgo', 'Pachuca de Soto')
print(df['Municipio'].unique())
#df = df[~df['Municipio'].isin(['Pachuca de Soto', 'Mineral de la Reforma', 'Zempoala'])]
#print(df['Municipio'].unique())

In [14]:
#df = df[df['Municipio'].isin(['Tulancingo de Bravo', 'Santiago Tulantepec de Lugo'])]


In [69]:
df['Estado'] = df['Estado'].replace('Sin_estado', 'Nayarit')
#df['Estado'] = df['Estado'].str.replace('jalisco', 'Jalisco')

print(df['Estado'].unique())

['Nayarit']


In [ ]:
print(df['Municipio'].unique())

In [ ]:
print(df[df['Municipio'] == 'Sin_municipio'].shape[0])

In [ ]:
print(df['Estado'].unique())
df['Estado'] = df['Estado'].replace('Sin_estado', 'Puebla')

df['Estado'] = df['Estado'].str.replace('querataro arteaga', 'queretaro')
df['Estado'] = df['Estado'].str.replace('Queretaro Arteaga', 'queretaro')
df['Estado'] = df['Estado'].str.replace('queretaro Arteaga', 'queretaro')
df['Estado'] = df['Estado'].str.replace('Santiago de Queretaro', 'queretaro')
df['Estado'] = df['Estado'].str.replace('Querétaro', 'queretaro')
df['Estado'] = df['Estado'].str.replace('Queretaro', 'queretaro')

print(df['Estado'].unique())

In [15]:
# Aplicando valor de acuerdo a la base a limpiar
#df['Estado'] = df['Estado'].replace('Sin_estado', 'Hidalgo')
df['Municipio'] = df['Municipio'].replace('sin_municipio', 'Puebla')


#### Estandarizar valores

In [70]:
# Minúsculas
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Estado'] = df['Estado'].str.strip().str.lower()

# Esta línea de código aplica la función unicodedata.normalize a la columna 'Colonia' del DataFrame df.
# El argumento 'NFKD' indica que se debe aplicar la normalización NFKD, que elimina los acentos y otros caracteres diacríticos.
#df['Colonia'] = df['Colonia'].apply(lambda x: unicodedata.normalize('NFKD', x))

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)
df['Municipio'] = df['Municipio'].apply(eliminar_acentos)
df['Estado'] = df['Estado'].apply(eliminar_acentos)
print(df['Colonia'].unique())

['mezcales' 'las jarretaderas' 'bucerias centro' 'san francisco'
 'cruz de juanacaxtle' 'nuevo vallarta' 'higuera blanca'
 'rincon del cielo' 'bahia de banderas' 'valle de banderas'
 'punta de mita' 'valle dorado' 'san vicente' 'las ceibas' 'terralta ii'
 'el porvenir' 'paraiso' 'altavela' 'vista bahia' 'sayulita' 'brisas'
 'villas de la bahia' 'playas de huanacaxtle' 'las palmas'
 'nuevo corral del risco' 'la mision' 'los arboles' 'costa coral'
 'jardines de san jose' 'lo de marcos' 'san juan de abajo'
 'jardines del sol' 'el mirador' 'santa fe' 'san jose del valle']


In [64]:
# Crear id de registros
df['id'] = range(1, len(df) + 1)

#### Separación de tipo de lugar del nombre de la colonia

In [71]:
# Añade las diferentes formas de identificar el tipo de lugar (etiquetas de ubicación)
tipos = ['condominio','fraccionamiento', 'infonavit', 'frac','fracc','fraccio','fracc.','namiento','ionamiento', 'uhp', 'urbanización','urbanizacion', 'sector', 'barrio', 'zona', 'sector', 'unidad habitacional','unidad hab']
#, 'residencial','unidad residencial'
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['tipo_lugar'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['tipo_lugar'], '').strip() if row['tipo_lugar'] else row['Colonia'], axis=1)
print(df[['Colonia','tipo_lugar','ubicacion']])
print(df['tipo_lugar'].unique())

                  Colonia tipo_lugar  \
0                mezcales       None   
1        las jarretaderas       None   
2         bucerias centro       None   
3           san francisco       None   
4     cruz de juanacaxtle       None   
...                   ...        ...   
1642    valle de banderas       None   
1643             mezcales       None   
1644    valle de banderas       None   
1645     rincon del cielo       None   
1646     jardines del sol       None   

                                            ubicacion  
0                Mezcales, Bahía de Banderas, Nayarit  
1        Las Jarretaderas, Bahía de Banderas, Nayarit  
2         Bucerías Centro, Bahía de Banderas, Nayarit  
3           San Francisco, Bahía de Banderas, Nayarit  
4     Cruz de Juanacaxtle, Bahía de Banderas, Nayarit  
...                                               ...  
1642    Valle de Banderas, Bahía de Banderas, Nayarit  
1643             Mezcales, Bahía de Banderas, Nayarit  
1644    Valle d

## c) Estandarizar valores (Mapeo)

### Diccionarios
1. Estandarizado
2. Base de códigos postales

#### 1 Diccionario personal, valores estandarizados

In [72]:
# Carga el diccionario de colonias estandarizadas desde un archivo CSV
diccionario = pd.read_csv('colonias_estandarizadas.csv')
print(diccionario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73814 entries, 0 to 73813
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ubicacion              73814 non-null  object 
 1   Colonia                73759 non-null  object 
 2   Municipio              73814 non-null  object 
 3   Estado                 73814 non-null  object 
 4   tipo_lugar             22825 non-null  object 
 5   Colonia_Estandarizada  73814 non-null  object 
 6   codigo_postal          73814 non-null  float64
dtypes: float64(1), object(6)
memory usage: 3.9+ MB
None


#### 2 Diccionario de base de codigos postales de la República Mexicana 

In [73]:
archivo_excel = 'CPdescarga.xlsx'
hojas_excel = pd.ExcelFile(archivo_excel, engine='openpyxl')
# Imprimir los nombres de las hojas
print(hojas_excel.sheet_names)

['Nota', 'Aguascalientes', 'Baja_California', 'Baja_California_Sur', 'Campeche', 'Coahuila_de_Zaragoza', 'Colima', 'Chiapas', 'Chihuahua', 'Distrito_Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán_de_Ocampo', 'Morelos', 'Nayarit', 'Nuevo_León', 'Querétaro', 'Oaxaca', 'Puebla', 'Quintana_Roo', 'Sonora', 'San_Luis_Potosí', 'Sinaloa', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz_de_Ignacio_de_la_Llave', 'Yucatán', 'Zacatecas']


CAMBIA DE ACUERDO AL LUGAR DE LA BASE

Un Estado

In [ ]:
# Obtener el DataFrame específico para los estados 'Jalisco' y 'Nayarit'
df_codigos_postales_jalisco = pd.read_excel(archivo_excel, sheet_name='Jalisco', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales_jalisco['estado'] = 'Jalisco'  # Agregar el nombre del estado como 'estado'

df_codigos_postales_nayarit = pd.read_excel(archivo_excel, sheet_name='Nayarit', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales_nayarit['estado'] = 'Nayarit'  # Agregar el nombre del estado como 'estado'
#df_codigos_postales_quintana_roo = pd.read_excel(archivo_excel, sheet_name='Quintana_Roo', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
#df_codigos_postales_quintana_roo['estado'] = 'Quintana_Roo'  # Agregar el nombre del estado como 'estado'

df_codigos_postales = pd.concat([df_codigos_postales_jalisco, df_codigos_postales_nayarit])
df_codigos_postales.head()

In [ ]:
df_codigos_postales['estado'].unique()

In [74]:
# Obtener el DataFrame específico para el estado 'Puebla'
df_codigos_postales = pd.read_excel(archivo_excel, sheet_name='Nayarit', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales['estado'] = 'Nayarit'  # Agregar el nombre del estado como 'estado'
df_codigos_postales.head()

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado
0,63000,Tepic Centro,Colonia,Tepic,Nayarit
1,63000,Esteban Baca Calderón,Unidad habitacional,Tepic,Nayarit
2,63010,Amado Nervo,Colonia,Tepic,Nayarit
3,63010,Amado Nervo,Fraccionamiento,Tepic,Nayarit
4,63010,Paseo de La Constitución,Fraccionamiento,Tepic,Nayarit


Si necesitas seleccionar un municipio en específico

In [ ]:
df_codigos_postales['D_mnpio'].unique()

In [ ]:
df_codigos_postales = df_codigos_postales[df_codigos_postales['D_mnpio'].str.contains('Puerto Vallarta|Bahía de Banderas', na=False)]
df_codigos_postales

In [75]:
# Limpiar y estandarizar las columnas
df_codigos_postales['D_mnpio'] = df_codigos_postales['D_mnpio'].str.lower().str.strip()
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].str.lower().str.strip()
df_codigos_postales['d_tipo_asenta'] = df_codigos_postales['d_tipo_asenta'].str.lower().str.strip()
df_codigos_postales['estado'] = df_codigos_postales['estado'].str.lower().str.strip()
# Eliminar acentos de todas las columnas de df_codigos_postales para que se pueda mapear bien
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', errors='ignore').decode('utf-8') if isinstance(x, str) else x)
df_codigos_postales['estado'] = df_codigos_postales['estado'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', errors='ignore').decode('utf-8') if isinstance(x, str) else x)
df_codigos_postales['D_mnpio'] = df_codigos_postales['D_mnpio'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', errors='ignore').decode('utf-8') if isinstance(x, str) else x)

In [76]:
df_codigos_postales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   d_codigo       2014 non-null   int64 
 1   d_asenta       2014 non-null   object
 2   d_tipo_asenta  2014 non-null   object
 3   D_mnpio        2014 non-null   object
 4   estado         2014 non-null   object
dtypes: int64(1), object(4)
memory usage: 78.8+ KB


In [113]:
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].fillna('N/A')
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].astype(str)



### Comparación

#### 1 Mapeo con Diccionario personal

De acuerdo a los estados y municipios exactos

In [77]:
def estand_col(colonia, municipio, estado, diccionario):
    # Busca en el diccionario si la colonia existe, considerando también el municipio y estado, y devuelve su nombre estandarizado
    # Si no existe, devuelve el nombre original de la colonia
    return diccionario[(diccionario['Colonia'] == colonia) & (diccionario['Municipio'] == municipio) & (diccionario['Estado'] == estado)]['Colonia_Estandarizada'].values[0] if (colonia, municipio, estado) in zip(diccionario['Colonia'], diccionario['Municipio'], diccionario['Estado']) else colonia

# Aplica la función de estandarización a la columna 'Colonia' del DataFrame df, considerando también 'Municipio' y 'Estado'
df['Colonia_Estandarizada'] = df.apply(lambda x: estand_col(x['Colonia'], x['Municipio'], x['Estado'], diccionario), axis=1)
df[['Colonia_Estandarizada','Colonia']]

,Colonia_Estandarizada,Colonia
0,mezcales,mezcales
1,las jarretaderas,las jarretaderas
2,bucerias centro,bucerias centro
3,san francisco,san francisco
4,cruz de huanacaxtle,cruz de juanacaxtle
...,...,...
1642,valle de banderas,valle de banderas
1643,mezcales,mezcales
1644,valle de banderas,valle de banderas
1645,rincon del cielo,rincon del cielo


De aceurdo a los nombres simples de colonia

In [78]:
#def estand_col(colonia, diccionario):
    # Busca en el diccionario si la colonia existe y devuelve su nombre estandarizado
    # Si no existe, devuelve el nombre original de la colonia
#    return diccionario[diccionario['Colonia'] == colonia]['Colonia_Estandarizada'].values[0] if colonia in diccionario['Colonia'].values else colonia

# Aplica la función de estandarización a la columna 'Colonia' del DataFrame df
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: estand_col(x, diccionario))
#df[['Colonia_Estandarizada','Colonia']]

In [79]:
# Función para obtener el código postal dado estado, municipio, y colonia
def obtener_codigo_postal(colonia, municipio, estado, df_codigos_postales):
     # Normalizar los inputs
    if pd.isna(colonia) or pd.isna(municipio) or pd.isna(estado):
        return None  # Devolver None si hay valores nulos

    # Normalizar los inputs
    estado = estado.lower().strip()
    municipio = municipio.lower().strip()
    colonia = colonia.lower().strip()
    
    estado = estado.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    municipio = municipio.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    colonia = colonia.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))

    # Filtrar el DataFrame según estado y municipio
    df_filtrado = df_codigos_postales[(df_codigos_postales['estado'] == estado) & (df_codigos_postales['D_mnpio'] == municipio)]
    
    # Intentar encontrar coincidencias exactas de la colonia
    resultado = df_filtrado[df_filtrado['d_asenta'] == colonia]
    
    if not resultado.empty:
        # Si se encuentra una coincidencia, devolver el código postal
        return resultado['d_codigo'].values[0]
    
    return None  # Devolver None si no hay coincidencia

In [80]:
# Aplicando valor de acuerdo a la base a limpiar
#df['Estado'] = df['Estado'].replace('Sin_estado', 'Hidalgo')
#df['Municipio'] = df['Municipio'].replace('sin_municipio', 'Pachuca de Soto')

In [81]:
df['codigo_postal'] = df.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado
sin_codigo_p = df[df['codigo_postal'].isna()]
Limpio = df[df['codigo_postal'].notna()]
print(sin_codigo_p[['Colonia','Municipio','Estado','codigo_postal']])
print("Número de registros sin código postal:", sin_codigo_p.shape[0])
print("Número de registros con código postal:", Limpio.shape[0])

      Colonia          Municipio   Estado  codigo_postal
139   paraiso  bahia de banderas  nayarit            NaN
202   paraiso  bahia de banderas  nayarit            NaN
234   paraiso  bahia de banderas  nayarit            NaN
237   paraiso  bahia de banderas  nayarit            NaN
811   paraiso  bahia de banderas  nayarit            NaN
1134  paraiso  bahia de banderas  nayarit            NaN
1424  paraiso  bahia de banderas  nayarit            NaN
Número de registros sin código postal: 7
Número de registros con código postal: 1603


#### 2 Mapeo con base, coincidencia flexible (fuzzy)

In [82]:
sin_codigo_p['Colonia'].unique()

array(['paraiso'], dtype=object)

In [83]:
# Función para mapear colonias con coincidencia flexible
def mapear_colonia_flexible(colonia, df_codigos_postales):
    # Convertir la colonia a string para evitar TypeError
    colonia_str = str(colonia)
    
    # Lista de todas las colonias en el diccionario
    colonias_diccionario = df_codigos_postales['d_asenta'].unique()

    # Buscar la coincidencia más cercana con fuzzywuzzy
    mejor_coincidencia = process.extractOne(colonia_str, colonias_diccionario, scorer=fuzz.ratio, score_cutoff=80)  # Ajusta el threshold de similitud a 80
    
    # Si se encuentra una coincidencia, devolverla; de lo contrario, devolver el valor original
    return mejor_coincidencia[0] if mejor_coincidencia else colonia_str
# Crear diccionario
# Aplicar la función al DataFrame original
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))
#df[['Colonia_Estandarizada','Colonia']]

# Con diccionario disponible
# Aplicar la función al DataFrame original
# Asegúrate de convertir la columna a string para evitar problemas
sin_codigo_p['Colonia'] = sin_codigo_p['Colonia'].astype(str)
sin_codigo_p['Colonia_Estandarizada'] = sin_codigo_p['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))
print(sin_codigo_p[['tipo_lugar', 'Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']])

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\600292204.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sin_codigo_p['Colonia'] = sin_codigo_p['Colonia'].astype(str)


     tipo_lugar  Colonia Colonia_Estandarizada          Municipio   Estado  \
139        None  paraiso               paraiso  bahia de banderas  nayarit   
202        None  paraiso               paraiso  bahia de banderas  nayarit   
234        None  paraiso               paraiso  bahia de banderas  nayarit   
237        None  paraiso               paraiso  bahia de banderas  nayarit   
811        None  paraiso               paraiso  bahia de banderas  nayarit   
1134       None  paraiso               paraiso  bahia de banderas  nayarit   
1424       None  paraiso               paraiso  bahia de banderas  nayarit   

                                ubicacion  
139   Paraíso, Bahía de Banderas, Nayarit  
202   Paraíso, Bahía de Banderas, Nayarit  
234   Paraíso, Bahía de Banderas, Nayarit  
237   Paraíso, Bahía de Banderas, Nayarit  
811   Paraíso, Bahía de Banderas, Nayarit  
1134  Paraíso, Bahía de Banderas, Nayarit  
1424  Paraíso, Bahía de Banderas, Nayarit  


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\600292204.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sin_codigo_p['Colonia_Estandarizada'] = sin_codigo_p['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))


In [84]:
#sin_codigo_p['codigo_postal'] = sin_codigo_p.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
sin_codigo_p.loc[:, 'codigo_postal'] = sin_codigo_p.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado

sin_codigo_postal = sin_codigo_p[sin_codigo_p['codigo_postal'].isna()]
Limpios = sin_codigo_p[sin_codigo_p['codigo_postal'].notna()]
print(sin_codigo_postal[['Colonia','Municipio','Estado','codigo_postal']].head())
print(sin_codigo_postal.shape)
print(Limpios.shape[0])

     Colonia          Municipio   Estado codigo_postal
139  paraiso  bahia de banderas  nayarit          None
202  paraiso  bahia de banderas  nayarit          None
234  paraiso  bahia de banderas  nayarit          None
237  paraiso  bahia de banderas  nayarit          None
811  paraiso  bahia de banderas  nayarit          None
(7, 40)
0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\4086569619.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[None None None None None None None]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sin_codigo_p.loc[:, 'codigo_postal'] = sin_codigo_p.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)


#### 3 Coincidencias parciales

In [85]:
# Mapear colonias de df_codigos_postales a las colonias de df para estandarizar

# Coincidencias exactas
#sin_codigo_postal['Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()].empty else x)
# Coincidencias parciales
sin_codigo_postal.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)
print(sin_codigo_postal[['Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(40))

      Colonia Colonia_Estandarizada          Municipio   Estado  \
139   paraiso    villas del paraiso  bahia de banderas  nayarit   
202   paraiso    villas del paraiso  bahia de banderas  nayarit   
234   paraiso    villas del paraiso  bahia de banderas  nayarit   
237   paraiso    villas del paraiso  bahia de banderas  nayarit   
811   paraiso    villas del paraiso  bahia de banderas  nayarit   
1134  paraiso    villas del paraiso  bahia de banderas  nayarit   
1424  paraiso    villas del paraiso  bahia de banderas  nayarit   

                                ubicacion  
139   Paraíso, Bahía de Banderas, Nayarit  
202   Paraíso, Bahía de Banderas, Nayarit  
234   Paraíso, Bahía de Banderas, Nayarit  
237   Paraíso, Bahía de Banderas, Nayarit  
811   Paraíso, Bahía de Banderas, Nayarit  
1134  Paraíso, Bahía de Banderas, Nayarit  
1424  Paraíso, Bahía de Banderas, Nayarit  


In [86]:
import pandas as pd

# Asegúrate de reemplazar valores NaN en las columnas relevantes
sin_codigo_postal['Colonia'] = sin_codigo_postal['Colonia'].fillna("").astype(str)
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].fillna("").astype(str)

# Función para buscar coincidencias exactas o parciales
def estandarizar_colonia(colonia, df_codigos_postales):
    if not colonia.strip():  # Si la colonia está vacía
        return "Sin información"
    try:
        # Coincidencia exacta
        match_exact = df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == colonia.lower()]
        if not match_exact.empty:
            return match_exact['d_asenta'].values[0]
        
        # Coincidencia parcial
        match_partial = df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(colonia, case=False, na=False)]
        if not match_partial.empty:
            return match_partial['d_asenta'].values[0]
        
        # Si no hay coincidencia, devolver la entrada original
        return colonia
    except Exception as e:
        print(f"Error procesando colonia '{colonia}': {e}")
        return colonia

# Aplicar la función al DataFrame
sin_codigo_postal['Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(
    lambda x: estandarizar_colonia(x, df_codigos_postales)
)

# Ver resultados
print(sin_codigo_postal[['Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']].head(40))


      Colonia Colonia_Estandarizada          Municipio   Estado  \
139   paraiso               paraiso  bahia de banderas  nayarit   
202   paraiso               paraiso  bahia de banderas  nayarit   
234   paraiso               paraiso  bahia de banderas  nayarit   
237   paraiso               paraiso  bahia de banderas  nayarit   
811   paraiso               paraiso  bahia de banderas  nayarit   
1134  paraiso               paraiso  bahia de banderas  nayarit   
1424  paraiso               paraiso  bahia de banderas  nayarit   

                                ubicacion  
139   Paraíso, Bahía de Banderas, Nayarit  
202   Paraíso, Bahía de Banderas, Nayarit  
234   Paraíso, Bahía de Banderas, Nayarit  
237   Paraíso, Bahía de Banderas, Nayarit  
811   Paraíso, Bahía de Banderas, Nayarit  
1134  Paraíso, Bahía de Banderas, Nayarit  
1424  Paraíso, Bahía de Banderas, Nayarit  


In [87]:
sin_codigo_postal.loc[:, 'codigo_postal'] = sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
#sin_codigo_postal['codigo_postal'] = sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].isna()]
Limpios2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].notna()]

print(sin_codigo_postal_2[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_2.shape)
print(Limpios2.shape)

      Colonia          Municipio   Estado codigo_postal
139   paraiso  bahia de banderas  nayarit          None
202   paraiso  bahia de banderas  nayarit          None
234   paraiso  bahia de banderas  nayarit          None
237   paraiso  bahia de banderas  nayarit          None
811   paraiso  bahia de banderas  nayarit          None
1134  paraiso  bahia de banderas  nayarit          None
1424  paraiso  bahia de banderas  nayarit          None
(7, 40)
(0, 40)


#### 4 Números a palabras + Coincidencias parciales

In [88]:
# Diccionario para convertir números cardinales a palabras
numeros_cardinales = {
    '1': 'uno',
    '2': 'dos',
    '3': 'tres',
    '4': 'cuatro',
    '5': 'cinco',
    '6': 'seis',
    '7': 'siete',
    '8': 'ocho',
    '9': 'nueve',
    '10': 'diez',
}

# Diccionario para convertir números ordinales a palabras
numeros_ordinales = {
    '1ra': 'primera',
    '2da': 'segunda',
    '3ra': 'tercera',
    '4ta': 'cuarta',
    '5ta': 'quinta',
    '6ta': 'sexta',
    '7ma': 'séptima',
    '8va': 'octava',
    '9na': 'novena',
    '10ma': 'décima',
    # Agregar más ordinales si es necesario
}
import re

def reemplazar_numeros(colonia):
    # Primero reemplazar los ordinales
    for num_ord, palabra_ord in numeros_ordinales.items():
        colonia = re.sub(rf'\b{num_ord}\b', palabra_ord, colonia)
    
    # Luego reemplazar los números cardinales aislados
    for num_card, palabra_card in numeros_cardinales.items():
        colonia = re.sub(rf'\b{num_card}\b', palabra_card, colonia)
    
    return colonia

# Utilizar .loc para asignar el resultado de la aplicación de la función reemplazar_numeros
sin_codigo_postal_2.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal_2['Colonia'].apply(reemplazar_numeros)
sin_codigo_postal_2['Colonia_Estandarizada'].unique()

array(['paraiso'], dtype=object)

In [90]:
sin_codigo_postal_2.loc[:, 'codigo_postal'] = sin_codigo_postal_2.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].isna()]
limpios3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].notna()]

print(sin_codigo_postal_3[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_3.shape)
print(limpios3.shape)

      Colonia          Municipio   Estado codigo_postal
139   paraiso  bahia de banderas  nayarit          None
202   paraiso  bahia de banderas  nayarit          None
234   paraiso  bahia de banderas  nayarit          None
237   paraiso  bahia de banderas  nayarit          None
811   paraiso  bahia de banderas  nayarit          None
1134  paraiso  bahia de banderas  nayarit          None
1424  paraiso  bahia de banderas  nayarit          None
(7, 40)
(0, 40)


#### 5 Limpieza manual, actualizar diccionario

In [91]:
df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3])
print(df_correctos.shape)
print(df_correctos.info())

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\1932133056.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3])


(1603, 40)
<class 'pandas.core.frame.DataFrame'>
Index: 1603 entries, 0 to 1646
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   precio                 1603 non-null   float64       
 1   denominacion           1603 non-null   object        
 2   propiedad              1603 non-null   object        
 3   metros_total           1603 non-null   int64         
 4   metros_construido      1603 non-null   int64         
 5   tiempo_de_publicacion  1603 non-null   object        
 6   tipo                   1603 non-null   object        
 7   estacionamientos       1603 non-null   int64         
 8   recamaras              1603 non-null   int64         
 9   banos                  1603 non-null   int64         
 10  medio_banos            1603 non-null   int64         
 11  seguridad_privada      1603 non-null   object        
 12  ubicacion              1603 non-null   object        
 1

In [92]:
registros_sin_codigo_postal = sin_codigo_postal_3[sin_codigo_postal_3['codigo_postal'].isna()]
shape_df = df.shape[0]
correct = df_correctos.shape[0]
sin_code = registros_sin_codigo_postal.shape[0]

print("Total de registros en DF original",f"{RED}{shape_df}{RESET}")
print("Registros sin código postal en df:", f'{MAGENTA}{sin_code}{RESET}')

print("Total de registros en df_correctos:", f'{BLUE}{correct}{RESET}')
print("Registros con código postal en df_correctos:", df_correctos[df_correctos['codigo_postal'].notna()].shape[0])
restantes = shape_df - correct
print("Restantes: ",f'{GREEN}{restantes}{RESET}')

Total de registros en DF original 1610
Registros sin código postal en df: 7
Total de registros en df_correctos: 1603
Registros con código postal en df_correctos: 1603
Restantes:  7


In [70]:
# Crear un diccionario para almacenar las estandarizaciones previas
# estandarizaciones_previas = {}
# Aplicar colores a los valores 
# def colorize_row(row):
#     row_colored = row.copy()  # Crear una copia para no modificar el original
#     row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
#     row_colored['Colonia_Estandarizada'] = f"{BLUE}{row['Colonia_Estandarizada']}{RESET}"
#     return row_colored
# Iterar sobre los registros sin código postal
# for idx, row in registros_sin_codigo_postal.iterrows():
#     print(f"Registro {idx}:")
#     print(colorize_row(row)[['ubicacion','Estado','Municipio','Colonia','Colonia_Estandarizada']])
#     # Verificar si ya existe una estandarización previa para la colonia actual
#     colonia_actual = row['Colonia']
#     if colonia_actual in estandarizaciones_previas:
#         # Si ya existe, usar el valor previo
#         new_value = estandarizaciones_previas[colonia_actual]
#         print(f"Usando estandarización previa para '{colonia_actual}': {new_value}")
#     else:
#         # Solicitar una nueva estandarización si no existe en el diccionario
#         new_value = input("Introduce estandarización de Colonia (presiona Enter para mantener el actual): ").lower()
#         if new_value == "":
#             new_value = colonia_actual  # Mantener el valor actual si no se ingresa nada
#         else:
#             estandarizaciones_previas[colonia_actual] = new_value  # Guardar la nueva estandarización
#     # Asignar el valor estandarizado al DataFrame
#     registros_sin_codigo_postal.at[idx, 'Colonia_Estandarizada'] = new_value
#     print("Cambio realizado por: ", new_value)
#     print("\n---\n")
# print("Proceso completado.")

##### Por si es queel estado o municipio han sido ingresados manualmente.

In [93]:
estandarizaciones_previas = {}  # Crear un diccionario para almacenar las estandarizaciones previas

for idx, row in registros_sin_codigo_postal.iterrows():
    print(f"Registro {idx}:")
    print(f"Ubicación: {MAGENTA}{row['ubicacion']}{RESET}")
    print(f"Estado: {BLUE}{row['Estado']}{RESET}")
    print(f"Municipio: {GREEN}{row['Municipio']}{RESET}")
    print(f"Colonia: {YELLOW}{row['Colonia']}{RESET}")
    colonia_actual = row['Colonia']
    if colonia_actual in estandarizaciones_previas:
        new_municipio = estandarizaciones_previas[colonia_actual]
        print(f"Usando estandarización previa para '{colonia_actual}': {new_municipio}")
    else:
        new_municipio = input("Introduce el nuevo municipio (presiona Enter para mantener el actual, 'b' para retroceder): ").lower()
        if new_municipio == "":
            new_municipio = row['Municipio']  # Mantener el valor actual si no se ingresa nada
        elif new_municipio == 'b':
            continue
        else:
            estandarizaciones_previas[colonia_actual] = new_municipio  # Guardar la nueva estandarización
    registros_sin_codigo_postal.at[idx, 'Municipio'] = new_municipio
print(f"{RED}Finalizado{RESET}")

Registro 139:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 202:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 234:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 237:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 811:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 1134:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Registro 1424:
Ubicación: Paraíso, Bahía de Banderas, Nayarit
Estado: nayarit
Municipio: bahia de banderas
Colonia: paraiso
Finalizado


#### Si no continuar..

In [94]:
registros_sin_codigo_postal_copia = registros_sin_codigo_postal.copy()

In [95]:
# Crear un diccionario para almacenar las estandarizaciones previas
estandarizaciones_previas = {}

# Aplicar colores a los valores 
def colorize_row(row):
    row_colored = row.copy()  
    row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
    row_colored['Colonia_Estandarizada'] = f"{BLUE}{row['Colonia_Estandarizada']}{RESET}"
    return row_colored

# Lista para llevar el historial de índices y estandarizaciones previas
historial_indices = []
idx_actual = 0  # Índice inicial

# Iterar sobre los registros sin código postal
while idx_actual < len(registros_sin_codigo_postal):
    idx = registros_sin_codigo_postal.index[idx_actual]
    row = registros_sin_codigo_postal.loc[idx]

    print(f"Registro {idx}:")
    print(colorize_row(row)[['ubicacion', 'Estado', 'Municipio', 'Colonia', 'Colonia_Estandarizada']])

    # Crear una clave única con el municipio y la colonia actual
    municipio_actual = row['Municipio']
    colonia_actual = row['Colonia']
    clave_unica = (municipio_actual, colonia_actual)
    
    # Verificar si ya existe una estandarización previa para la combinación exacta de municipio y colonia
    if clave_unica in estandarizaciones_previas:
        # Si ya existe, usar el valor previo
        new_value = estandarizaciones_previas[clave_unica]
        print(f"Usando estandarización previa para '{colonia_actual}' en '{municipio_actual}': {new_value}")
    else:
        # Solicitar una nueva estandarización si no existe en el diccionario
        while True:
            print(f"Colonia a estandarizar: {colonia_actual}")
            new_value = input(f"Introduce estandarización de '{colonia_actual}' (presiona Enter para mantener el actual, 'b' para retroceder): ").lower()
            if new_value == "":
                new_value = colonia_actual  # Mantener el valor actual si no se ingresa nada
                break
            elif new_value == 'b':
                # Retroceder al registro anterior si hay historial disponible
                if historial_indices:
                    # Revertir al último índice y valor estandarizado
                    idx_actual = historial_indices.pop()  # Volver al índice anterior
                    idx = registros_sin_codigo_postal.index[idx_actual]
                    # Obtener la clave del registro anterior y eliminarla del diccionario si ya se guardó
                    municipio_anterior = registros_sin_codigo_postal.at[idx, 'Municipio']
                    colonia_anterior = registros_sin_codigo_postal.at[idx, 'Colonia']
                    clave_anterior = (municipio_anterior, colonia_anterior)
                    if clave_anterior in estandarizaciones_previas:
                        del estandarizaciones_previas[clave_anterior]
                    break
                else:
                    print("No hay registro anterior para retroceder.")
            else:
                estandarizaciones_previas[clave_unica] = new_value  # Guardar la nueva estandarización
                break
                
    # Aplicar el cambio solo si es necesario
    if registros_sin_codigo_postal.at[idx, 'Colonia_Estandarizada'] != new_value:
        registros_sin_codigo_postal.at[idx, 'Colonia_Estandarizada'] = new_value
        print("Cambio realizado:", new_value)

    # Guardar el índice actual en el historial si no es un retroceso
    historial_indices.append(idx_actual)
    idx_actual += 1  # Avanzar al siguiente registro

print(f"{GREEN}Proceso completado.{RESET}")

# No aplicar drop_duplicates para no perder registros originales
print("Registros únicos en base original conservados. Modificaciones aplicadas únicamente a 'Colonia_Estandarizada'.")


Registro 139:
ubicacion                Paraíso, Bahía de Banderas, Nayarit
Estado                                                        nayarit
Municipio                                           bahia de banderas
Colonia                                                       paraiso
Colonia_Estandarizada                                paraiso
Name: 139, dtype: object
Colonia a estandarizar: paraiso
Cambio realizado: paraíso vallarta
Registro 202:
ubicacion                Paraíso, Bahía de Banderas, Nayarit
Estado                                                        nayarit
Municipio                                           bahia de banderas
Colonia                                                       paraiso
Colonia_Estandarizada                                paraiso
Name: 202, dtype: object
Usando estandarización previa para 'paraiso' en 'bahia de banderas': paraíso vallarta
Cambio realizado: paraíso vallarta
Registro 234:
ubicacion                Paraíso, Bahía de Banderas, Nay

In [ ]:
# Crear un diccionario para almacenar las estandarizaciones previas
# estandarizaciones_previas = {}
# Aplicar colores a los valores 
# def colorize_row(row):
#     row_colored = row.copy()  # Crear una copia para no modificar el original
#     row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
#     row_colored['Colonia_Estandarizada'] = f"{BLUE}{row['Colonia_Estandarizada']}{RESET}"
#     return row_colored
# Iterar sobre los registros sin código postal
# idx_anterior = None
# for idx, row in registros_sin_codigo_postal.iterrows():
#     print(f"Registro {idx}:")
#     print(colorize_row(row)[['ubicacion','Estado','Municipio','Colonia','Colonia_Estandarizada']])
#     # Verificar si ya existe una estandarización previa para la colonia actual
#     colonia_actual = row['Colonia']
#     if colonia_actual in estandarizaciones_previas:
#         # Si ya existe, usar el valor previo
#         new_value = estandarizaciones_previas[colonia_actual]
#         print(f"Usando estandarización previa para '{colonia_actual}': {new_value}")
#     else:
#         # Solicitar una nueva estandarización si no existe en el diccionario
#         while True:
#             new_value = input("Introduce estandarización de Colonia (presiona Enter para mantener el actual, 'b' para retroceder): ").lower()
#             if new_value == "":
#                 new_value = colonia_actual  # Mantener el valor actual si no se ingresa nada
#                 break
#             elif new_value == 'b':
#                 if idx_anterior is not None:
#                     idx = idx_anterior
#                     break
#                 else:
#                     print("No hay registro anterior para retroceder.")
#             else:
#                 estandarizaciones_previas[colonia_actual] = new_value  # Guardar la nueva estandarización
#                 break
#     idx_anterior = idx
#     # Asignar el valor estandarizado al DataFrame
#     registros_sin_codigo_postal.at[idx, 'Colonia_Estandarizada'] = new_value
#     print("Cambio realizado por: ", new_value)
#     print("\n---\n")
# print("Proceso completado.")

In [96]:
#registros_sin_codigo_postal['codigo_postal'] = registros_sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
registros_sin_codigo_postal.loc[:, 'codigo_postal'] = registros_sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado
no_code = registros_sin_codigo_postal[registros_sin_codigo_postal['codigo_postal'].isna()]
clean = registros_sin_codigo_postal[registros_sin_codigo_postal['codigo_postal'].notna()]
print(no_code[['Colonia','Municipio','Estado','codigo_postal']])
print(no_code.shape)
print(clean.shape)

      Colonia          Municipio   Estado codigo_postal
139   paraiso  bahia de banderas  nayarit          None
202   paraiso  bahia de banderas  nayarit          None
234   paraiso  bahia de banderas  nayarit          None
237   paraiso  bahia de banderas  nayarit          None
811   paraiso  bahia de banderas  nayarit          None
1134  paraiso  bahia de banderas  nayarit          None
1424  paraiso  bahia de banderas  nayarit          None
(7, 40)
(0, 40)


In [ ]:
estandarizaciones_previas = {}  # Crear un diccionario para almacenar las estandarizaciones previas

for idx, row in no_code.iterrows():
    print(f"Registro {idx}:")
    print(f"Ubicación: {MAGENTA}{row['ubicacion']}{RESET}")
    print(f"Estado: {BLUE}{row['Estado']}{RESET}")
    print(f"Municipio: {GREEN}{row['Municipio']}{RESET}")
    print(f"Colonia: {YELLOW}{row['Colonia']}{RESET}")
    colonia_actual = row['Colonia']
    if colonia_actual in estandarizaciones_previas:
        new_municipio = estandarizaciones_previas[colonia_actual]
        print(f"Usando estandarización previa para '{colonia_actual}': {new_municipio}")
    else:
        new_municipio = input("Introduce el nuevo municipio (presiona Enter para mantener el actual, 'b' para retroceder): ").lower()
        if new_municipio == "":
            new_municipio = row['Municipio']  # Mantener el valor actual si no se ingresa nada
        elif new_municipio == 'b':
            continue
        else:
            estandarizaciones_previas[colonia_actual] = new_municipio  # Guardar la nueva estandarización
    no_code.at[idx, 'Municipio'] = new_municipio

In [ ]:

#no_code['codigo_postal'] = no_code.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
no_code.loc[:, 'codigo_postal'] = no_code.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado
no_code_2 = no_code[no_code['codigo_postal'].isna()]
clean2 = no_code[no_code['codigo_postal'].notna()]
print(no_code_2.shape)
print(clean2.shape)

In [ ]:
for col in clean.columns:
    print(f'{col}: {clean[col].map(type).unique()}')


In [97]:
clean = clean.drop(columns=['baños_desc'])
#df_correctos = df_correctos.drop(columns=['baños_desc'])

In [98]:
clean = clean.drop_duplicates(keep='first')
##clean = pd.concat([clean, clean2]).drop_duplicates(keep='first')
shapes = clean.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

La base mantuvo una estructura de:  (0, 39)


In [99]:
if 'baños_desc' in df_correctos.columns:
    df_correctos['baños_desc'] = df_correctos['baños_desc'].astype(str)
if 'baños_desc' in clean.columns:
    clean['baños_desc'] = clean['baños_desc'].astype(str)
df_correctos = pd.concat([df_correctos,clean]).drop_duplicates(keep='first')
print(df_correctos.shape)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\606738327.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_correctos = pd.concat([df_correctos,clean]).drop_duplicates(keep='first')


(1603, 40)


In [100]:
registros_sin_codigo_postal_fin = registros_sin_codigo_postal[registros_sin_codigo_postal['codigo_postal'].isna()]
shape_df = df.shape[0]
correct = df_correctos.shape[0]
sin_code = registros_sin_codigo_postal_fin.shape[0]

print("Total de registros en DF original",f"{RED}{shape_df}{RESET}")
print("Registros sin código postal en df:", f'{MAGENTA}{sin_code}{RESET}')

print("Total de registros en df_correctos:", f'{BLUE}{correct}{RESET}')
print("Registros con código postal en df_correctos:", df_correctos[df_correctos['codigo_postal'].notna()].shape[0])
restantes = shape_df - correct
print("Restantes: ",f'{GREEN}{restantes}{RESET}')

Total de registros en DF original 1610
Registros sin código postal en df: 7
Total de registros en df_correctos: 1603
Registros con código postal en df_correctos: 1603
Restantes:  7


##### Diccionario

In [101]:
# Crear diccionario de 0
#df_correctos[['ubicacion','Colonia','Municipio', 'Estado','tipo_lugar','Colonia_Estandarizada', 'codigo_postal']].to_csv('colonias_estandarizadas.csv', index=False, header=True, mode='w', encoding='utf-8')

In [102]:
# Identificar colonias estandarizadas que no se repiten o ya se encuentran estandarizadas en el diccionario
colonias_no_en_diccionario = df_correctos[df_correctos['Colonia_Estandarizada'].apply(lambda x: x not in diccionario)].copy()
# Crear un DataFrame de nuevas entradas sin duplicados, manteniendo las columnas del DataFrame original
nuevas_entradas = colonias_no_en_diccionario[['ubicacion','Colonia','Municipio', 'Estado','tipo_lugar','Colonia_Estandarizada', 'codigo_postal']].drop_duplicates()

# Actualizar el diccionario con las nuevas colonias sin reemplazar las existentes
# Excluir entradas vacías antes de la concatenación para evitar el FutureWarning
nuevas_entradas_sin_vacios = nuevas_entradas[nuevas_entradas['Colonia_Estandarizada'].notna()]
diccionario = pd.concat([diccionario, nuevas_entradas_sin_vacios]).drop_duplicates(keep='first')
diccionario

,ubicacion,Colonia,Municipio,Estado,tipo_lugar,Colonia_Estandarizada,codigo_postal
0,"Las Aves, Puebla, Puebla",las aves,puebla,puebla,NaN,las aves,72582.0
1,"Estación Nueva, Puebla, Puebla",estacion nueva,puebla,puebla,NaN,estacion nueva,72223.0
2,"Lomas Flor Del Bosque, Puebla, Puebla",lomas flor del bosque,puebla,puebla,NaN,lomas flor del bosque,72360.0
3,"Santa Mónica, Puebla, Puebla",santa monica,puebla,puebla,NaN,santa monica,72540.0
4,"Villa Satélite la Calera, Puebla, Puebla",villa satelite la calera,puebla,puebla,NaN,villa satelite calera,72564.0
...,...,...,...,...,...,...,...
68758,"Nuevo Corral del Risco, Bahía de Banderas, Nayarit",nuevo corral del risco,bahia de banderas,nayarit,NaN,corral del risco (punta de mita),63734.0
71283,"El Mirador, Bahía de Banderas, Nayarit",el mirador,bahia de banderas,nayarit,NaN,el mirador,63730.0
71284,"Tule Dorado, Bahía de Banderas, Nayarit",tule dorado,bahia de banderas,nayarit,NaN,tule dorado,63732.0
71285,"San Juan de Abajo, Bahía de Banderas, Nayarit",san juan de abajo,bahia de banderas,nayarit,NaN,san juan de abajo,63730.0


In [103]:
# Exportar el diccionario actualizado al archivo original, pero sin sobrescribirlo
# En su lugar, agregar las nuevas entradas como nuevos registros
diccionario.to_csv('colonias_estandarizadas.csv', index=False, header=False, mode='a', encoding='utf-8')

#### 6 Asignar códigos postales manualmente

In [104]:
# Crear un diccionario para almacenar los códigos postales previos
codigos_postales_previos = {}

# Iterar sobre los registros sin código postal
for idx, row in registros_sin_codigo_postal_fin.iterrows():
    # Extraer la información relevante de cada registro
    colonia_estandarizada_actual = row['Colonia_Estandarizada']
    estado_actual = row['Estado']
    municipio_actual = row['Municipio']
    colonia_actual = row['Colonia']

    # Si el estado es 'Puerto Vallarta', cambiarlo a 'Jalisco'
    if estado_actual == 'Puerto Vallarta':
        estado_actual = 'Jalisco'

    # Imprimir la información del registro actual
    print(f"{MAGENTA}Estado: {estado_actual}{RESET}, {MAGENTA}Municipio: {municipio_actual}{RESET}, "
          f"{MAGENTA}Colonia: {colonia_actual}{RESET}, {MAGENTA}Colonia Estandarizada: {colonia_estandarizada_actual}{RESET}")

    # Crear una clave única para almacenar el código postal basado en estado, municipio, y colonia estandarizada
    clave_unica = (estado_actual, municipio_actual, colonia_estandarizada_actual)

    # Verificar si ya existe un código postal previo para la combinación exacta
    if clave_unica in codigos_postales_previos:
        # Si ya existe, usar el valor previo
        new_codigo_postal = codigos_postales_previos[clave_unica]
        print(f"Usando código postal previo para '{colonia_estandarizada_actual}' en '{municipio_actual}, {estado_actual}': {new_codigo_postal}")
    else:
        # Solicitar un nuevo código postal si no existe en el diccionario
        while True:
            try:
                new_codigo_postal = int(input(f"{BLUE}Introduce código postal para '{colonia_estandarizada_actual}' en '{municipio_actual}, {estado_actual}': {RESET}"))
                break
            except ValueError:
                print("Por favor, ingrese un código postal válido (sólo números enteros).")
        # Guardar el nuevo código postal en el diccionario con la clave única
        codigos_postales_previos[clave_unica] = new_codigo_postal

    # Asignar el código postal al DataFrame
    registros_sin_codigo_postal_fin.at[idx, 'codigo_postal'] = new_codigo_postal
    print(f"{GREEN}Cambio realizado por: {new_codigo_postal}{RESET}")
    print("\n---\n")

print("Proceso completado.")

Estado: nayarit, Municipio: bahia de banderas, Colonia: paraiso, Colonia Estandarizada: paraíso vallarta
Cambio realizado por: 63735

---

Estado: nayarit, Municipio: bahia de banderas, Colonia: paraiso, Colonia Estandarizada: paraíso vallarta
Usando código postal previo para 'paraíso vallarta' en 'bahia de banderas, nayarit': 63735
Cambio realizado por: 63735

---

Estado: nayarit, Municipio: bahia de banderas, Colonia: paraiso, Colonia Estandarizada: paraíso vallarta
Usando código postal previo para 'paraíso vallarta' en 'bahia de banderas, nayarit': 63735
Cambio realizado por: 63735

---

Estado: nayarit, Municipio: bahia de banderas, Colonia: paraiso, Colonia Estandarizada: paraíso vallarta
Usando código postal previo para 'paraíso vallarta' en 'bahia de banderas, nayarit': 63735
Cambio realizado por: 63735

---

Estado: nayarit, Municipio: bahia de banderas, Colonia: paraiso, Colonia Estandarizada: paraíso vallarta
Usando código postal previo para 'paraíso vallarta' en 'bahia de b

In [105]:
registros_sin_codigo_postal_fin = registros_sin_codigo_postal_fin[registros_sin_codigo_postal_fin['codigo_postal'] != 0]
registros_sin_codigo_postal_fin.shape

(7, 40)

Identifica que coincidan los datos, para que la estandarización no te haya hecho perder ningún registro

In [106]:
finales_sin_code = registros_sin_codigo_postal_fin[registros_sin_codigo_postal_fin['codigo_postal'].isna()]
finales_con_code = registros_sin_codigo_postal_fin[registros_sin_codigo_postal_fin['codigo_postal'].notna()]
df_correctos_final = pd.concat([df_correctos, finales_con_code])
#df_correctos_final.drop_duplicates(inplace=True)

shape_df = df.shape[0]
correctos_finales = df_correctos_final.shape[0]
sin_code_final = finales_sin_code.shape[0]
print("Total de registros en DF original",f"{RED}{shape_df}{RESET}")
print("Registros sin código postal en df:", f'{MAGENTA}{sin_code_final}{RESET}')

print("Total de registros en df_correctos:", f'{BLUE}{correctos_finales}{RESET}')
print("Registros con código postal en df_correctos:", df_correctos_final[df_correctos_final['codigo_postal'].notna()].shape[0])
restantes = shape_df - correctos_finales
print("Restantes: ",f'{GREEN}{restantes}{RESET}')

# Identificar los registros extra en df_correctos en comparación con df
registros_extra = df_correctos_final[~df_correctos_final.index.isin(df.index)]
print("Registros extra en df_correctos en comparación con df:")
print(registros_extra)

Total de registros en DF original 1610
Registros sin código postal en df: 0
Total de registros en df_correctos: 1610
Registros con código postal en df_correctos: 1610
Restantes:  0
Registros extra en df_correctos en comparación con df:
Empty DataFrame
Columns: [precio, denominacion, propiedad, metros_total, metros_construido, tiempo_de_publicacion, tipo, estacionamientos, recamaras, banos, medio_banos, seguridad_privada, ubicacion, url, descripcion, precio_desc, precio_mxn, precio_usd, fecha_conversion, categoria, fecha_estandarizada, dias_transcurridos, meses_transcurridos, fecha_mas_actual, totales_desc, construidos_desc, precio_m2_terreno, precio_m2_construido, estacionamiento_desc, recamaras_desc, baños_desc, baños_completos, medios_baños, bano_total, Colonia, Municipio, Estado, tipo_lugar, Colonia_Estandarizada, codigo_postal]
Index: []

[0 rows x 40 columns]


In [ ]:
finales_sin_code

In [ ]:
df.columns

In [107]:
df_shape = df.shape[0]
correctos = df_correctos_final.shape[0]

print(f"{BLUE}ORIGINAL: {df_shape}{RESET}")
print(f"{GREEN}LIMPIOS FINALES: {correctos}{RESET}")

ORIGINAL: 1610
LIMPIOS FINALES: 1610


____
# 13. Variables oficiales

In [108]:
df = df_correctos_final
df.columns

Index(['precio', 'denominacion', 'propiedad', 'metros_total',
       'metros_construido', 'tiempo_de_publicacion', 'tipo',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos',
       'seguridad_privada', 'ubicacion', 'url', 'descripcion', 'precio_desc',
       'precio_mxn', 'precio_usd', 'fecha_conversion', 'categoria',
       'fecha_estandarizada', 'dias_transcurridos', 'meses_transcurridos',
       'fecha_mas_actual', 'totales_desc', 'construidos_desc',
       'precio_m2_terreno', 'precio_m2_construido', 'estacionamiento_desc',
       'recamaras_desc', 'baños_desc', 'baños_completos', 'medios_baños',
       'bano_total', 'Colonia', 'Municipio', 'Estado', 'tipo_lugar',
       'Colonia_Estandarizada', 'codigo_postal'],
      dtype='object')

In [ ]:
df = df.drop(['denominacion', 'precio_desc','baños_desc','baños_completos','medios_baños'], axis=1, errors='ignore') #'fecha_conversion'

In [109]:
# Tiempo de publicación limpio:
df['tiempo_de_publicacion'] = df['fecha_estandarizada']
# Categoría limpia
df['categoria'].fillna('', inplace=True)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3220\3915492155.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['categoria'].fillna('', inplace=True)


EXTRA

In [110]:
df['Status'] = 'Venta'
df['id'] = range(1, len(df) + 1)

ORDEN OFICIAL

In [ ]:
columnas_ordenadas = ['id', 'categoria', 'precio','precio_mxn','precio_usd','fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
                      'precio_m2_terreno', 'precio_m2_construido','tiempo_de_publicacion', 'dias_transcurridos', 'meses_transcurridos','fecha_mas_actual', 'tipo','Status',
                      'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total','seguridad_privada', 'tipo_lugar','Colonia','Municipio','Estado','Colonia_Estandarizada','codigo_postal','ubicacion','url','descripcion']
df = df[columnas_ordenadas]

In [195]:
columnas_ordenadas = ['id', 'categoria', 'precio','precio_mxn','precio_usd','fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
                      'precio_m2_terreno', 'precio_m2_construido','tiempo_de_publicacion', 'dias_transcurridos', 'meses_transcurridos','fecha_mas_actual', 'tipo','Status',
                      'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total','seguridad_privada', 'Colonia','Estado','url']#'Colonia_Estandarizada','codigo_postal','tipo_lugar','Municipio','ubicacion',
df = df[columnas_ordenadas] # 'descripcion' commented out


In [ ]:
df.info()

# 14. Exportar

Cambia el nombre por el que desees en 'archivo'

In [111]:
import os

archivo = 'lamudi_BahiaBanderas_Septiembre_2024'  # Nombre de archivo a exportar
#lamudi-Bahia-de-Balderas_Inmuebles_Septiembre 2024
ext = '.csv'  # Extensión del archivo
ciudad = 'Vallarta'  # Nombre de la ciudad en minúsculcva

ruta = os.path.join('../../data', ciudad, 'clean')
if not os.path.exists(ruta):
    os.makedirs(ruta)

archivo_export = os.path.join(ruta, archivo + ext)

# Exportar el DataFrame a la ruta y archivo creados
df.to_csv(archivo_export, encoding='utf-8', index=False)
print(f"Archivo guardado en: {archivo_export}")

Archivo guardado en: ../../data\Vallarta\clean\lamudi_BahiaBanderas_Agosto_2024.csv


In [ ]:
import os

archivo = 'nombre_del_archivo'  # Nombre de archivo a exportar
ext = '.csv'  # Extensión del archivo
ciudad = ''  # Nombre de la ciudad en minúsculcva

ruta = os.path.join('..', 'ciudades', ciudad, 'clean')
if not os.path.exists(ruta):
    os.makedirs(ruta)

archivo_export = os.path.join(ruta, archivo + ext)

# Exportar el DataFrame a la ruta y archivo creados
df.to_csv(archivo_export, encoding='utf-8', index=False)
print(f"Archivo guardado en: {archivo_export}")